# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [1]:

import tequila as tq
from utility import * #you will need this utility.py from week2
import time

In [2]:
#Running with cell goes over all possible optinons  
molecules=['h2','h4','lih','h2o','n2','nh3']
methods=['hf','cisd','ccsd','fci']
basies=['sto-3g','6-31g']
qubit_transfms=['jw','bk'] 

In [3]:
#Fcus on one molecule
molecules=['h2']
methods=['cisd']
basies=['sto-3g']
qubit_transfms=['jw'] #Jrdan-Wigner transformation is better

In [4]:
R=0.8 #bond_length

for mol in molecules:

    for basis in basies:

        for qubit_transf in qubit_transfms:
                    
            start=time.time()
            
            H = get_qubit_hamiltonian(mol, geometry=R, basis=basis, qubit_transf=qubit_transf)
            print("Take a look at H!")
            
            xyz_data = get_molecular_data(mol, geometry=R, xyz_format=True)
            Htq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

            E_cisd=obtain_PES(mol, [R], basis, 'cisd')
            print('cisd'.upper()," ground state energy is:",E_cisd)
            
            n_spin_orbitals=2*Htq.n_orbitals
            n_electrons=Htq.n_electrons
            print("Numbers of electrons:",n_electrons,
                  "\nUsing ",qubit_transf.upper()," in ",basis.upper(),"basis.",
                  "\nNumber of spin-orbitals (qubits):",n_spin_orbitals,
                  "\nNumber of Hamiltonian terms:",len(H.terms)) 

            H_eff=taper_hamiltonian(H, n_spin_orbitals, n_electrons, qubit_transf)
            print("Number of terms in Taper effective Hamiltonian (H_eff):",len(H_eff.terms))

            
            if mol=='h2':
                # Building the matrix representation of the effective Hamiltonian
                I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
                H_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

                # Obtain the eigenvalues
                eigvals, _ = np.linalg.eigh(H_matrix)
                print("The eigenvalues of the effective Hamiltonian:{}".format(eigvals))
            else:
                print("H_matrix for qubit-tapering technique not yet implemeneted for ",mol.upper())

            print(mol.upper(),"Hamiltonian using ",qubit_transf.upper()," in basis:",
                  basis.upper(),"generated for:",time.time()-start," sec\n")
            
            

Take a look at H!
r=0.8000, E =  -1.13415 Eh
CISD  ground state energy is: [-1.13414767]
Numbers of electrons: 2 
Using  JW  in  STO-3G basis. 
Number of spin-orbitals (qubits): 4 
Number of Hamiltonian terms: 15
Number of terms in Taper effective Hamiltonian (H_eff): 3
The eigenvalues of the effective Hamiltonian:[-1.10115031  0.03904763]
H2 Hamiltonian using  JW  in basis: STO-3G generated for: 1.4550960063934326  sec



In [5]:
H

-0.16733398905695201 [] +
-0.04615669588901533 [X0 X1 Y2 Y3] +
0.04615669588901533 [X0 Y1 Y2 X3] +
0.04615669588901533 [Y0 X1 X2 Y3] +
-0.04615669588901533 [Y0 Y1 X2 X3] +
0.16251648748871636 [Z0] +
0.16583253721590413 [Z0 Z1] +
0.11720364720195853 [Z0 Z2] +
0.16336034309097386 [Z0 Z3] +
0.16251648748871636 [Z1] +
0.16336034309097386 [Z1 Z2] +
0.11720364720195853 [Z1 Z3] +
-0.19744293699755833 [Z2] +
0.1716978839228672 [Z2 Z3] +
-0.19744293699755827 [Z3]

In [6]:
H_eff

-0.39093154850404543 [] +
0.18462678355606132 [X0] +
-0.7199188489725494 [Z0]

In [7]:
qwc_list = get_qwc_group(H)
print('Fragments 1: \n{}\n'.format(qwc_list[0]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.16733398905695201 [] +
0.04615669588901533 [X0 Y1 Y2 X3]

Fragments 2:
-0.04615669588901533 [X0 X1 Y2 Y3]

Number of fragments: 5


In [8]:
comm_groups = get_commuting_group(H)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])
print(comm_groups[2])

Number of mutually commuting fragments: 2
The first commuting group
-0.16733398905695201 [] +
-0.04615669588901533 [X0 X1 Y2 Y3] +
0.04615669588901533 [X0 Y1 Y2 X3] +
0.04615669588901533 [Y0 X1 X2 Y3] +
-0.04615669588901533 [Y0 Y1 X2 X3] +
0.16583253721590413 [Z0 Z1] +
0.11720364720195853 [Z0 Z2] +
0.16336034309097386 [Z0 Z3] +
0.16336034309097386 [Z1 Z2] +
0.11720364720195853 [Z1 Z3] +
0.1716978839228672 [Z2 Z3]
0.16251648748871636 [Z0] +
0.16251648748871636 [Z1] +
-0.19744293699755833 [Z2] +
-0.19744293699755827 [Z3]


In [9]:
def imp_form4(allz):
    new_form={}
    for keys in allz.terms:
        eeee=list('eeee')
        for key in keys:
            eeee[key[0]]="z"
        new_form["".join(eeee)]=allz.terms[keys]
    return new_form 

In [10]:
def imp_form(allz,n_qubits): #n_spin_orbitals
    new_form={}
    for keys in allz.terms:
        eeee=['e' for i in range(n_qubits)]
        for key in keys:
            eeee[key[0]]="z"
        new_form["".join(eeee)]=allz.terms[keys]
    return new_form 

In [11]:
%%time
#One has to loop over all groups

new_form=[]
for i in range(1,len(comm_groups)+1):
    uqwc = get_qwc_unitary(comm_groups[i])
    qwc = remove_complex(uqwc * comm_groups[i] * uqwc)

    uz = get_zform_unitary(qwc)
    print("Checking whether U * U^+ is identity: {}".format(uz * uz))

    allz = remove_complex(uz * qwc * uz)
    print("\nThe all-z form of qwc fragment:\n{}".format(allz))
    new_form.append(imp_form(allz,n_spin_orbitals))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.16733398905695185 [] +
0.1633603430909737 [Z0] +
0.16583253721590394 [Z0 Z1] +
0.04615669588901527 [Z0 Z1 Z3] +
0.11720364720195839 [Z0 Z2] +
-0.04615669588901527 [Z0 Z3] +
0.11720364720195839 [Z1] +
0.1633603430909737 [Z1 Z2] +
-0.04615669588901527 [Z1 Z2 Z3] +
0.17169788392286692 [Z2] +
0.04615669588901527 [Z2 Z3]
Checking whether U * U^+ is identity: 0.9999999999999996 []

The all-z form of qwc fragment:
0.16251648748871614 [Z0] +
0.16251648748871614 [Z1] +
-0.1974429369975582 [Z2] +
-0.19744293699755802 [Z3]
CPU times: user 53.1 ms, sys: 90 µs, total: 53.2 ms
Wall time: 52.2 ms


In [12]:
new_form

[{'ezzz': -0.04615669588901527,
  'eezz': 0.04615669588901527,
  'zzez': 0.04615669588901527,
  'zeez': -0.04615669588901527,
  'eeze': 0.17169788392286692,
  'ezee': 0.11720364720195839,
  'ezze': 0.1633603430909737,
  'zeee': 0.1633603430909737,
  'zeze': 0.11720364720195839,
  'zzee': 0.16583253721590394,
  'eeee': -0.16733398905695185},
 {'eeez': -0.19744293699755802,
  'eeze': -0.1974429369975582,
  'ezee': 0.16251648748871614,
  'zeee': 0.16251648748871614}]

In [13]:
n_lements=0
all_togeter={}
for group in new_form:
    print(4,len(group),'real')
    for elemet in group:
        print(elemet,group[elemet])
        if elemet in all_togeter:
            temp=all_togeter[elemet]
            temp+=group[elemet]
            all_togeter[elemet]=temp
        else:
            n_lements+=1
            temp=group[elemet]
            all_togeter[elemet]=temp

print()        
print(n_spin_orbitals,n_lements,'real',"R=",R,"E[Eh]=",E_cisd[0]," Use the lines below only!")
for key in all_togeter:
    print(key,all_togeter[key])

4 11 real
ezzz -0.04615669588901527
eezz 0.04615669588901527
zzez 0.04615669588901527
zeez -0.04615669588901527
eeze 0.17169788392286692
ezee 0.11720364720195839
ezze 0.1633603430909737
zeee 0.1633603430909737
zeze 0.11720364720195839
zzee 0.16583253721590394
eeee -0.16733398905695185
4 4 real
eeez -0.19744293699755802
eeze -0.1974429369975582
ezee 0.16251648748871614
zeee 0.16251648748871614

4 12 real R= 0.8 E[Eh]= -1.1341476666428472  Use the lines below only!
ezzz -0.04615669588901527
eezz 0.04615669588901527
zzez 0.04615669588901527
zeez -0.04615669588901527
eeze -0.02574505307469127
ezee 0.2797201346906745
ezze 0.1633603430909737
zeee 0.32587683057968986
zeze 0.11720364720195839
zzee 0.16583253721590394
eeee -0.16733398905695185
eeez -0.19744293699755802


In [14]:
-0.3796856633927857+0.15183385960079301

-0.22785180379199269

In [15]:
allz.__dict__

{'terms': {((3, 'Z'),): -0.19744293699755802,
  ((2, 'Z'),): -0.1974429369975582,
  ((1, 'Z'),): 0.16251648748871614,
  ((0, 'Z'),): 0.16251648748871614}}

In [16]:
imp_form(allz,n_spin_orbitals)

{'eeez': -0.19744293699755802,
 'eeze': -0.1974429369975582,
 'ezee': 0.16251648748871614,
 'zeee': 0.16251648748871614}

In [17]:
#This is E0 at R=1 for h2 using H_eff
-0.531051349433764-0.5350572998841723

-1.0661086493179361